In [1]:
from itertools import product
import gurobipy as gp
import pandas as pd
from openpyxl import load_workbook
import os
import random
from datetime import datetime
from itertools import product

In [2]:
print(gp.gurobi.version())

(10, 0, 1)


In [3]:
current_time = datetime.now()
timestamp = current_time.timestamp()
random.seed(int(timestamp))

In [4]:
# 0. 读⼊⽂件
data_1 = pd.read_excel(r"C:\Users\86181\Desktop\数模作业、笔记\农作物种植策略\附件1修改版.xlsx", sheet_name="乡村的现有耕地")
data_2 = pd.read_excel(r"C:\Users\86181\Desktop\数模作业、笔记\农作物种植策略\附件2修改版.xlsx", sheet_name="2023年的农作物种植情况")
data_2 = data_2.ffill()# 处理合并单元格的问题
data_3 = pd.read_excel(r"C:\Users\86181\Desktop\数模作业、笔记\农作物种植策略\附件2修改版.xlsx", sheet_name="2023年统计的相关数据")
trans_dict = {"A": "平旱地", "B": "梯田", "C": "山坡地", "D": "水浇地", "E": "普通大棚", "F": "智慧大棚"}#创建字典键值对

In [5]:
print(trans_dict)

{'A': '平旱地', 'B': '梯田', 'C': '山坡地', 'D': '水浇地', 'E': '普通大棚', 'F': '智慧大棚'}


In [6]:
print(data_1)

   地块名称   地块类型  地块面积/亩                                                说明 
0    A1    平旱地    80.0  (1) 平旱地、梯田和山坡地每年都只能种植一季作物。\n\n(2) 水浇地每年可以种植一季也...
1    A2    平旱地    55.0                                                NaN
2    A3    平旱地    35.0                                                NaN
3    A4    平旱地    72.0                                                NaN
4    A5    平旱地    68.0                                                NaN
5    A6    平旱地    55.0                                                NaN
6    B1     梯田    60.0                                                NaN
7    B2     梯田    46.0                                                NaN
8    B3     梯田    40.0                                                NaN
9    B4     梯田    28.0                                                NaN
10   B5     梯田    25.0                                                NaN
11   B6     梯田    86.0                                                NaN
12   B7     梯田    55.0                

In [7]:
print(data_2)

   种植地块  作物编号 作物名称    作物类型  种植面积/亩 种植季次
0    A1     6   小麦      粮食    80.0   单季
1    A2     7   玉米      粮食    55.0   单季
2    A3     7   玉米      粮食    35.0   单季
3    A4     1   黄豆  粮食（豆类）    72.0   单季
4    A5     4   绿豆  粮食（豆类）    68.0   单季
..  ...   ...  ...     ...     ...  ...
82   F3    28  小青菜     蔬菜      0.3  第二季
83   F3    30  生菜      蔬菜      0.3  第二季
84   F4    19   芸豆  蔬菜（豆类）     0.6  第一季
85   F4    34   芹菜     蔬菜      0.3  第二季
86   F4    23  菠菜      蔬菜      0.3  第二季

[87 rows x 6 columns]


In [8]:
import pandas as pd

# 假设 data_3 是一个 DataFrame
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('display.width', None)  # 自动调整宽度

print(data_3)


      序号  作物编号 作物名称   地块类型 种植季次  亩产量/斤  种植成本/(元/亩)    销售单价/(元/斤)
0      1     1   黄豆    平旱地   单季    400         400     2.50-4.00
1      2     2   黑豆    平旱地   单季    500         400     6.50-8.50
2      3     3   红豆    平旱地   单季    400         350     7.50-9.00
3      4     4   绿豆    平旱地   单季    350         350     6.00-8.00
4      5     5   爬豆    平旱地   单季    415         350     6.00-7.50
5      6     6   小麦    平旱地   单季    800         450     3.00-4.00
6      7     7   玉米    平旱地   单季   1000         500     2.50-3.50
7      8     8   谷子    平旱地   单季    400         360     6.00-7.50
8      9     9   高粱    平旱地   单季    630         400     5.50-6.50
9     10    10   黍子    平旱地   单季    525         360     6.50-8.50
10    11    11   荞麦    平旱地   单季    110         350   30.00-50.00
11    12    12   南瓜    平旱地   单季   3000        1000     1.00-2.00
12    13    13   红薯    平旱地   单季   2200        2000     2.50-4.00
13    14    14   莜麦    平旱地   单季    420         400     5.00-6.00
14    15    15   大麦    平旱

In [9]:
# 1. 输⼊参数
# 1.1 销量
q_0 = {(data_2["种植地块"][i], data_2["作物编号"][i],
        2 if data_2["种植季次"][i] == "第二季" else 1):
            data_2['种植面积/亩'][i]
        for i in range(len(data_2))}

In [10]:
print(q_0)

{('A1', 6, 1): 80.0, ('A2', 7, 1): 55.0, ('A3', 7, 1): 35.0, ('A4', 1, 1): 72.0, ('A5', 4, 1): 68.0, ('A6', 8, 1): 55.0, ('B1', 6, 1): 60.0, ('B2', 2, 1): 46.0, ('B3', 3, 1): 40.0, ('B4', 4, 1): 28.0, ('B5', 5, 1): 25.0, ('B6', 8, 1): 86.0, ('B7', 6, 1): 55.0, ('B8', 8, 1): 44.0, ('B9', 9, 1): 50.0, ('B10', 10, 1): 25.0, ('B11', 1, 1): 60.0, ('B12', 7, 1): 45.0, ('B13', 14, 1): 35.0, ('B14', 15, 1): 20.0, ('C1', 11, 1): 15.0, ('C2', 12, 1): 13.0, ('C3', 1, 1): 15.0, ('C4', 13, 1): 18.0, ('C5', 6, 1): 27.0, ('C6', 3, 1): 20.0, ('D1', 20, 1): 15.0, ('D1', 36, 2): 15.0, ('D2', 28, 1): 10.0, ('D2', 35, 2): 10.0, ('D3', 21, 1): 14.0, ('D3', 35, 2): 14.0, ('D4', 22, 1): 6.0, ('D4', 35, 2): 6.0, ('D5', 17, 1): 10.0, ('D5', 36, 2): 10.0, ('D6', 18, 1): 12.0, ('D6', 37, 2): 12.0, ('D7', 16, 1): 22.0, ('D8', 16, 1): 20.0, ('E1', 18, 1): 0.6, ('E1', 38, 2): 0.6, ('E2', 24, 1): 0.6, ('E2', 38, 2): 0.6, ('E3', 25, 1): 0.6, ('E3', 38, 2): 0.6, ('E4', 26, 1): 0.6, ('E4', 39, 2): 0.6, ('E5', 28, 1): 0

In [11]:
q = {}
for (i, j, k), v in q_0.items():
    key = (trans_dict[i[0]], j, k)
    if key in q:
        q[key] += v
    else:
        q[key] = v
    q[key] = round(q[key], 1)

In [12]:
print(q)#q的键是地块、作物编号、季度的元组，值是对应的面积，其实就是销量

{('平旱地', 6, 1): 80.0, ('平旱地', 7, 1): 90.0, ('平旱地', 1, 1): 72.0, ('平旱地', 4, 1): 68.0, ('平旱地', 8, 1): 55.0, ('梯田', 6, 1): 115.0, ('梯田', 2, 1): 46.0, ('梯田', 3, 1): 40.0, ('梯田', 4, 1): 28.0, ('梯田', 5, 1): 25.0, ('梯田', 8, 1): 130.0, ('梯田', 9, 1): 50.0, ('梯田', 10, 1): 25.0, ('梯田', 1, 1): 60.0, ('梯田', 7, 1): 45.0, ('梯田', 14, 1): 35.0, ('梯田', 15, 1): 20.0, ('山坡地', 11, 1): 15.0, ('山坡地', 12, 1): 13.0, ('山坡地', 1, 1): 15.0, ('山坡地', 13, 1): 18.0, ('山坡地', 6, 1): 27.0, ('山坡地', 3, 1): 20.0, ('水浇地', 20, 1): 15.0, ('水浇地', 36, 2): 25.0, ('水浇地', 28, 1): 10.0, ('水浇地', 35, 2): 30.0, ('水浇地', 21, 1): 14.0, ('水浇地', 22, 1): 6.0, ('水浇地', 17, 1): 10.0, ('水浇地', 18, 1): 12.0, ('水浇地', 37, 2): 12.0, ('水浇地', 16, 1): 42.0, ('普通大棚', 18, 1): 1.2, ('普通大棚', 38, 2): 1.8, ('普通大棚', 24, 1): 0.6, ('普通大棚', 25, 1): 0.6, ('普通大棚', 26, 1): 0.6, ('普通大棚', 39, 2): 1.8, ('普通大棚', 28, 1): 0.6, ('普通大棚', 27, 1): 0.9, ('普通大棚', 19, 1): 1.2, ('普通大棚', 40, 2): 1.8, ('普通大棚', 17, 1): 1.2, ('普通大棚', 41, 2): 4.2, ('普通大棚', 22, 1): 0.6, ('普通大棚', 21, 1)

In [13]:
data_3["地块类型"] = data_3["地块类型"].astype(str)
data_4 = {(data_3["地块类型"][i].strip(), data_3["作物编号"][i],
            2 if data_3["种植季次"][i] == "第二季" else 1):
                [data_3["亩产量/斤"][i], data_3["种植成本/(元/亩)"][i], data_3["销售单价/(元/斤)"][i]]
            for i in range(len(data_3))}
# 1.2 亩产， 成本和价格
#r、c、p的键是地块类型、作物编号、季度组成的元组
#r的值是亩产，c的值是成本，p的值是价格
_, r, c, p = gp.multidict(data_4)#调用gurobipy，从一个数据源创建多个字典
pl = {}
pu = {}
for k, v in p.items():
    try:
        str1 = v.split('-')#把价格区间的两端分别复制给pl、pu
        pl[k] = float(str1[0])
        pu[k] = float(str1[1])
    except:
        pass
    #pl是price_low;pu是price_up
# 1.3 地块⾯积
a = {data_1["地块名称"][i]: data_1["地块面积/亩"][i]
    for i in range(len(data_1))}

In [14]:
print({key: value for key, value in data_4.items()})


{('平旱地', 1, 1): [400, 400, '2.50-4.00'], ('平旱地', 2, 1): [500, 400, '6.50-8.50'], ('平旱地', 3, 1): [400, 350, '7.50-9.00'], ('平旱地', 4, 1): [350, 350, '6.00-8.00'], ('平旱地', 5, 1): [415, 350, '6.00-7.50'], ('平旱地', 6, 1): [800, 450, '3.00-4.00'], ('平旱地', 7, 1): [1000, 500, '2.50-3.50'], ('平旱地', 8, 1): [400, 360, '6.00-7.50'], ('平旱地', 9, 1): [630, 400, '5.50-6.50'], ('平旱地', 10, 1): [525, 360, '6.50-8.50'], ('平旱地', 11, 1): [110, 350, '30.00-50.00'], ('平旱地', 12, 1): [3000, 1000, '1.00-2.00'], ('平旱地', 13, 1): [2200, 2000, '2.50-4.00'], ('平旱地', 14, 1): [420, 400, '5.00-6.00'], ('平旱地', 15, 1): [525, 350, '3.00-4.00'], ('梯田', 1, 1): [380, 400, '2.50-4.00'], ('梯田', 2, 1): [475, 400, '6.50-8.50'], ('梯田', 3, 1): [380, 350, '7.50-9.00'], ('梯田', 4, 1): [330, 350, '6.00-8.00'], ('梯田', 5, 1): [395, 350, '6.00-7.50'], ('梯田', 6, 1): [760, 450, '3.00-4.00'], ('梯田', 7, 1): [950, 500, '2.50-3.50'], ('梯田', 8, 1): [380, 360, '6.00-7.50'], ('梯田', 9, 1): [600, 400, '5.50-6.50'], ('梯田', 10, 1): [500, 360, '6.50-8.5

In [15]:
# 2. 定义集合
# 2.1 作物
LDR = ["黄豆", "黑豆", "红豆", "绿豆", "爬豆"] # 粮⻝（⾖类）
LD = list(range(1, 6))
LR = ["小麦", "玉米", "谷子", "高粱", "黍子", "荞麦", "南瓜", "红薯", "莜麦", "大麦"] # 粮⻝
L = list(range(6, 16))
LSR = ["水稻"] # 粮⻝⽔浇地
LS = list(range(16, 17))
SDR = ["豇豆", "刀豆", "芸豆"] # 蔬菜（⾖类）
SD = list(range(17, 20))
S_1R = ["土豆", "西红柿", "茄子", "菠菜", "青椒", "菜花", "包菜", "油麦菜", "小青菜","黄瓜", "生菜", "辣椒", "空心菜", "黄心菜", "芹菜"]
S_1 = list(range(20, 35))
S_2R = ["大白菜", "白萝卜", "红萝卜"]
S_2 = list(range(35, 38))
SJR = ["榆黄菇", "香菇", "白灵菇", "羊肚菌"]
SJ = list(range(38, 42))
J = LD + L + LS + SD + S_1 + S_2 + SJ#作物编号集合
J1 = LDR + LR + LSR + SDR + S_1R + S_2R + SJR#作物集合
J_d = dict(zip(J1, J))#J1作为键，J作为值
# 2.2 地块
A = ["A" + str(i) for i in range(1, 7)] # 得到形如“A1，A2...A6”的列表
B = ["B" + str(i) for i in range(1, 15)] # 梯⽥
C = ["C" + str(i) for i in range(1, 7)] # ⼭坡地
D = ["D" + str(i) for i in range(1, 9)] # ⽔浇地
E = ["E" + str(i) for i in range(1, 17)] # 普通⼤棚
F = ["F" + str(i) for i in range(1, 5)] # 智慧⼤棚
trans_dict_2 = {"平旱地": A, "梯田": B, "山坡地": C, "水浇地": D, "普通大棚": E,"智慧大棚": F}
I = A + B + C + D + E + F#地块集合
# 2.3 年份
Y = list(range(2024, 2031))
# 2.4 季节
KR = ["第一季", "第二季"]
K = [1, 2]
# 2.5 集合组合
#按种植耕地分
#product计算笛卡尔积,V是四维变量
V_1 = list(product(A + B + C, LD + L, [K[0]], Y))
V_2 = list(product(D, LS, [K[0]], Y))
V_3 = list(product(D + E + F, SD + S_1, [K[0]], Y))
V_4 = list(product(F, SD + S_1, [K[1]], Y))#智慧大棚的第二季单独出来
V_5 = list(product(D, S_2, [K[1]], Y))
V_6 = list(product(E, SJ, [K[1]], Y))
V = V_1 + V_2 + V_3 + V_4 + V_5 + V_6
#按季节来分
#W是三维变量，表示地块季节集合，与作物无关，用于面积约束
W1 = product(A + B + C + D + E + F, [K[0]], Y)
W2 = product(D + E + F, [K[1]], Y)
W = list(W1) + list(W2)
#地块类型集合
II = list(trans_dict.values())


In [16]:
print(Y)

[2024, 2025, 2026, 2027, 2028, 2029, 2030]


In [17]:
print(V_1)

[('A1', 1, 1, 2024), ('A1', 1, 1, 2025), ('A1', 1, 1, 2026), ('A1', 1, 1, 2027), ('A1', 1, 1, 2028), ('A1', 1, 1, 2029), ('A1', 1, 1, 2030), ('A1', 2, 1, 2024), ('A1', 2, 1, 2025), ('A1', 2, 1, 2026), ('A1', 2, 1, 2027), ('A1', 2, 1, 2028), ('A1', 2, 1, 2029), ('A1', 2, 1, 2030), ('A1', 3, 1, 2024), ('A1', 3, 1, 2025), ('A1', 3, 1, 2026), ('A1', 3, 1, 2027), ('A1', 3, 1, 2028), ('A1', 3, 1, 2029), ('A1', 3, 1, 2030), ('A1', 4, 1, 2024), ('A1', 4, 1, 2025), ('A1', 4, 1, 2026), ('A1', 4, 1, 2027), ('A1', 4, 1, 2028), ('A1', 4, 1, 2029), ('A1', 4, 1, 2030), ('A1', 5, 1, 2024), ('A1', 5, 1, 2025), ('A1', 5, 1, 2026), ('A1', 5, 1, 2027), ('A1', 5, 1, 2028), ('A1', 5, 1, 2029), ('A1', 5, 1, 2030), ('A1', 6, 1, 2024), ('A1', 6, 1, 2025), ('A1', 6, 1, 2026), ('A1', 6, 1, 2027), ('A1', 6, 1, 2028), ('A1', 6, 1, 2029), ('A1', 6, 1, 2030), ('A1', 7, 1, 2024), ('A1', 7, 1, 2025), ('A1', 7, 1, 2026), ('A1', 7, 1, 2027), ('A1', 7, 1, 2028), ('A1', 7, 1, 2029), ('A1', 7, 1, 2030), ('A1', 8, 1, 2024),

In [18]:
# 3. 处理参数

# 3.1 销量处理
sale_data = {}
#q是面积，r是亩产，乘起来就是销量
sale_data_2023 = {i: q[i] * r[i]
                    for i in q}#i遍历q的每个键，i是地块、作物编号、季度的元组
#sale_data_2023的键是地块、作物编号、季度的元组，值是销量

In [19]:
print(q)

{('平旱地', 6, 1): 80.0, ('平旱地', 7, 1): 90.0, ('平旱地', 1, 1): 72.0, ('平旱地', 4, 1): 68.0, ('平旱地', 8, 1): 55.0, ('梯田', 6, 1): 115.0, ('梯田', 2, 1): 46.0, ('梯田', 3, 1): 40.0, ('梯田', 4, 1): 28.0, ('梯田', 5, 1): 25.0, ('梯田', 8, 1): 130.0, ('梯田', 9, 1): 50.0, ('梯田', 10, 1): 25.0, ('梯田', 1, 1): 60.0, ('梯田', 7, 1): 45.0, ('梯田', 14, 1): 35.0, ('梯田', 15, 1): 20.0, ('山坡地', 11, 1): 15.0, ('山坡地', 12, 1): 13.0, ('山坡地', 1, 1): 15.0, ('山坡地', 13, 1): 18.0, ('山坡地', 6, 1): 27.0, ('山坡地', 3, 1): 20.0, ('水浇地', 20, 1): 15.0, ('水浇地', 36, 2): 25.0, ('水浇地', 28, 1): 10.0, ('水浇地', 35, 2): 30.0, ('水浇地', 21, 1): 14.0, ('水浇地', 22, 1): 6.0, ('水浇地', 17, 1): 10.0, ('水浇地', 18, 1): 12.0, ('水浇地', 37, 2): 12.0, ('水浇地', 16, 1): 42.0, ('普通大棚', 18, 1): 1.2, ('普通大棚', 38, 2): 1.8, ('普通大棚', 24, 1): 0.6, ('普通大棚', 25, 1): 0.6, ('普通大棚', 26, 1): 0.6, ('普通大棚', 39, 2): 1.8, ('普通大棚', 28, 1): 0.6, ('普通大棚', 27, 1): 0.9, ('普通大棚', 19, 1): 1.2, ('普通大棚', 40, 2): 1.8, ('普通大棚', 17, 1): 1.2, ('普通大棚', 41, 2): 4.2, ('普通大棚', 22, 1): 0.6, ('普通大棚', 21, 1)

In [20]:
print(r)

{('平旱地', 1, 1): 400, ('平旱地', 2, 1): 500, ('平旱地', 3, 1): 400, ('平旱地', 4, 1): 350, ('平旱地', 5, 1): 415, ('平旱地', 6, 1): 800, ('平旱地', 7, 1): 1000, ('平旱地', 8, 1): 400, ('平旱地', 9, 1): 630, ('平旱地', 10, 1): 525, ('平旱地', 11, 1): 110, ('平旱地', 12, 1): 3000, ('平旱地', 13, 1): 2200, ('平旱地', 14, 1): 420, ('平旱地', 15, 1): 525, ('梯田', 1, 1): 380, ('梯田', 2, 1): 475, ('梯田', 3, 1): 380, ('梯田', 4, 1): 330, ('梯田', 5, 1): 395, ('梯田', 6, 1): 760, ('梯田', 7, 1): 950, ('梯田', 8, 1): 380, ('梯田', 9, 1): 600, ('梯田', 10, 1): 500, ('梯田', 11, 1): 105, ('梯田', 12, 1): 2850, ('梯田', 13, 1): 2100, ('梯田', 14, 1): 400, ('梯田', 15, 1): 500, ('山坡地', 1, 1): 360, ('山坡地', 2, 1): 450, ('山坡地', 3, 1): 360, ('山坡地', 4, 1): 315, ('山坡地', 5, 1): 375, ('山坡地', 6, 1): 720, ('山坡地', 7, 1): 900, ('山坡地', 8, 1): 360, ('山坡地', 9, 1): 570, ('山坡地', 10, 1): 475, ('山坡地', 11, 1): 100, ('山坡地', 12, 1): 2700, ('山坡地', 13, 1): 2000, ('山坡地', 14, 1): 380, ('山坡地', 15, 1): 475, ('水浇地', 16, 1): 500, ('水浇地', 17, 1): 3000, ('水浇地', 18, 1): 2000, ('水浇地', 19, 1): 3000, ('

In [21]:
print(('智慧大棚', 32, 2) in q)  # 应该输出 True
print(('智慧大棚', 32, 2) in r)  # 应该输出 True

False
True


In [22]:
'''不需要销量处理
for idx, t in enumerate(Y):#t遍历年份，idx是元素的索引，从0开始
    # i: ⼟地类型， j: 作物, k: 季节
    # 统⼀：random_number = random.uniform(-0.05, 0.05)
    for i, j, k in q:
        random_number = random.uniform(-0.05, 0.05)
        if idx == 0:
            sale_data[i, j, k, t] = sale_data_2023[i, j, k] * (1 +random_number)
        else:
            sale_data[i, j, k, t] = sale_data[i, j, k, Y[idx - 1]] * (1 +random_number)
        
    # ⼩⻨ 和 ⽟⽶ 未来 的 预期销售量有增⻓的趋势，平均年增⻓率介于
    for i, j, k in q:
        if j not in [J_d["⼩⻨"], J_d["⽟⽶"]]:
            continue
    random_number = random.uniform(0.05, 0.1)#在规定范围内生成随机数
    if idx == 0:
        sale_data[i, j, k, t] = sale_data_2023[i, j, k] * (1 +random_number)
    else:
        sale_data[i, j, k, t] = sale_data[i, j, k, Y[idx - 1]] * (1 +random_number)
'''
# 3.2 亩产处理
mu_data = {}
mu_data_2023 = {i: r[i]
                for i in r}
'''不需要亩产处理
for idx, t in enumerate(Y):
    # i: ⼟地类型， j: 作物, k: 季节
    # 统⼀：random_number = random.uniform(-0.05, 0.05)
    for i, j, k in r:
        random_number = random.uniform(-0.1, 0.1)
        if idx == 0:
            mu_data[i, j, k, t] = mu_data_2023[i, j, k] * (1 + random_number)
        else:
            mu_data[i, j, k, t] = mu_data[i, j, k, Y[idx - 1]] * (1 +random_number)
'''
# 3.3 成本处理
ch_data = {}
ch_data_2023 = {i: c[i]
                for i in c}
'''
for idx, t in enumerate(Y):
    # i: ⼟地类型， j: 作物, k: 季节
    # 统⼀：random_number = random.uniform(-0.05, 0.05)
    for i, j, k in c:
        random_number = random.uniform(0, 0.05)
        if idx == 0:
            ch_data[i, j, k, t] = ch_data_2023[i, j, k] * (1 + random_number)
        else:
            ch_data[i, j, k, t] = ch_data[i, j, k, Y[idx - 1]] * (1 +random_number)
'''
# 3.4 价格处理
jia_data_l = {}
jia_data_2023_l = {i: pl[i]
                for i in pl}
jia_data_u = {}
jia_data_2023_u = {i: pu[i]
                for i in pl}


In [23]:
for idx, t in enumerate(Y):#t遍历年份，idx是元素的索引，从0开始
    for i, j, k in q:
        sale_data[i, j, k, t] = sale_data_2023[i, j, k]
    for i, j, k in q:
        sale_data[i, j, k, t] = sale_data_2023[i, j, k]
    for i, j, k in r:
        mu_data[i, j, k, t] = mu_data_2023[i, j, k]
    for i, j, k in c:
        ch_data[i, j, k, t] = ch_data_2023[i, j, k]
    for i, j, k in pl:
        jia_data_l[i, j, k, t] = jia_data_2023_l[i, j, k]
        jia_data_u[i, j, k, t] = jia_data_2023_u[i, j, k]

In [24]:
# 4. 建⽴模型
m = gp.Model("cumcm_2024")

In [25]:
# 4.1 决策变量
x = m.addVars(V, vtype=gp.GRB.CONTINUOUS, lb=0, name="x")
y = m.addVars(V, vtype=gp.GRB.BINARY, lb=0, name="y")
x_s = m.addVars(pl, Y, vtype=gp.GRB.CONTINUOUS, lb=0, name="xs")
l_s = m.addVars(pl, Y, vtype=gp.GRB.BINARY, name="ls")

In [26]:
# 4.2 约束条件
# 1. ⾯积约束

m.addConstrs((x[i, j, k, t] <= a[i] * y[i, j, k, t]
for i, j, k, t in V), "C11")
m.addConstrs((x.sum(i, "*", k, t) <= a[i]
for (i, k, t) in W), "C12")

{('A1', 1, 2024): <gurobi.Constr *Awaiting Model Update*>,
 ('A1', 1, 2025): <gurobi.Constr *Awaiting Model Update*>,
 ('A1', 1, 2026): <gurobi.Constr *Awaiting Model Update*>,
 ('A1', 1, 2027): <gurobi.Constr *Awaiting Model Update*>,
 ('A1', 1, 2028): <gurobi.Constr *Awaiting Model Update*>,
 ('A1', 1, 2029): <gurobi.Constr *Awaiting Model Update*>,
 ('A1', 1, 2030): <gurobi.Constr *Awaiting Model Update*>,
 ('A2', 1, 2024): <gurobi.Constr *Awaiting Model Update*>,
 ('A2', 1, 2025): <gurobi.Constr *Awaiting Model Update*>,
 ('A2', 1, 2026): <gurobi.Constr *Awaiting Model Update*>,
 ('A2', 1, 2027): <gurobi.Constr *Awaiting Model Update*>,
 ('A2', 1, 2028): <gurobi.Constr *Awaiting Model Update*>,
 ('A2', 1, 2029): <gurobi.Constr *Awaiting Model Update*>,
 ('A2', 1, 2030): <gurobi.Constr *Awaiting Model Update*>,
 ('A3', 1, 2024): <gurobi.Constr *Awaiting Model Update*>,
 ('A3', 1, 2025): <gurobi.Constr *Awaiting Model Update*>,
 ('A3', 1, 2026): <gurobi.Constr *Awaiting Model Update*

In [27]:
 # 2. ⽔稻每年⼀季节或者两季节蔬菜

m.addConstrs((y[i, LS[0], K[0], t] + y[i, j, K[0], t] <= 1
 for i in D
 for j in S_1 + SD
 for t in Y), "C21")


 m.addConstrs((y[i, LS[0], K[0], t] + y[i, j, K[1], t] <= 1
for i in D
for j in S_2
for t in Y), "C22")

{('D1', 35, 2024): <gurobi.Constr *Awaiting Model Update*>,
 ('D1', 35, 2025): <gurobi.Constr *Awaiting Model Update*>,
 ('D1', 35, 2026): <gurobi.Constr *Awaiting Model Update*>,
 ('D1', 35, 2027): <gurobi.Constr *Awaiting Model Update*>,
 ('D1', 35, 2028): <gurobi.Constr *Awaiting Model Update*>,
 ('D1', 35, 2029): <gurobi.Constr *Awaiting Model Update*>,
 ('D1', 35, 2030): <gurobi.Constr *Awaiting Model Update*>,
 ('D1', 36, 2024): <gurobi.Constr *Awaiting Model Update*>,
 ('D1', 36, 2025): <gurobi.Constr *Awaiting Model Update*>,
 ('D1', 36, 2026): <gurobi.Constr *Awaiting Model Update*>,
 ('D1', 36, 2027): <gurobi.Constr *Awaiting Model Update*>,
 ('D1', 36, 2028): <gurobi.Constr *Awaiting Model Update*>,
 ('D1', 36, 2029): <gurobi.Constr *Awaiting Model Update*>,
 ('D1', 36, 2030): <gurobi.Constr *Awaiting Model Update*>,
 ('D1', 37, 2024): <gurobi.Constr *Awaiting Model Update*>,
 ('D1', 37, 2025): <gurobi.Constr *Awaiting Model Update*>,
 ('D1', 37, 2026): <gurobi.Constr *Await

In [28]:
# 3. 重种约束
m.addConstrs((y[i, j, K[0], Y[t]] + y[i, j, K[0], Y[t + 1]] <= 1
for i in A + B + C
for j in LD + L
for t in range(len(Y) - 1)), "C31")

m.addConstrs((y[i, j, K[0], Y[t]] + y[i, j, K[0], Y[t + 1]] <= 1
for i in D
for j in LS
for t in range(len(Y) - 1)), "C32")


m.addConstrs((y[i, j, K[0], Y[t]] + y[i, j, K[1], Y[t]] <= 1
for i in F
for j in SD + S_1
for t in range(len(Y))), "C33")

m.addConstrs((y[i, j, K[1], Y[t]] + y[i, j, K[0], Y[t + 1]] <= 1
for i in F
for j in SD + S_1
for t in range(len(Y) - 1)), "C34")

{('F1', 17, 0): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 17, 1): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 17, 2): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 17, 3): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 17, 4): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 17, 5): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 18, 0): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 18, 1): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 18, 2): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 18, 3): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 18, 4): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 18, 5): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 19, 0): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 19, 1): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 19, 2): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 19, 3): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 19, 4): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 19, 5): <gurobi.Constr 

In [29]:
# 4. 所有⼟地三年内⾄少种植⼀次⾖类作物
m.addConstrs((gp.quicksum(y[i, j, K[0], Y[tau]]
for j in LD
for tau in range(t, t + 3)) >= 1
for i in A + B + C
for t in range(0, len(Y) - 2)), "C41")

m.addConstrs((gp.quicksum(y[i, j, K[0], Y[tau]]
for j in SD
for tau in range(t, t + 3)) >= 1
for i in D + E
for t in range(0, len(Y) - 2)), "C42")

m.addConstrs((gp.quicksum(y[i, j, k, Y[tau]]
for k in K
for j in SD
for tau in range(t, t + 3)) >= 1
for i in F
for t in range(0, len(Y) - 2)), "C43")

{('F1', 0): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 1): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 2): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 3): <gurobi.Constr *Awaiting Model Update*>,
 ('F1', 4): <gurobi.Constr *Awaiting Model Update*>,
 ('F2', 0): <gurobi.Constr *Awaiting Model Update*>,
 ('F2', 1): <gurobi.Constr *Awaiting Model Update*>,
 ('F2', 2): <gurobi.Constr *Awaiting Model Update*>,
 ('F2', 3): <gurobi.Constr *Awaiting Model Update*>,
 ('F2', 4): <gurobi.Constr *Awaiting Model Update*>,
 ('F3', 0): <gurobi.Constr *Awaiting Model Update*>,
 ('F3', 1): <gurobi.Constr *Awaiting Model Update*>,
 ('F3', 2): <gurobi.Constr *Awaiting Model Update*>,
 ('F3', 3): <gurobi.Constr *Awaiting Model Update*>,
 ('F3', 4): <gurobi.Constr *Awaiting Model Update*>,
 ('F4', 0): <gurobi.Constr *Awaiting Model Update*>,
 ('F4', 1): <gurobi.Constr *Awaiting Model Update*>,
 ('F4', 2): <gurobi.Constr *Awaiting Model Update*>,
 ('F4', 3): <gurobi.Constr *Awaiting Model Upd

In [30]:
# 5. 种植地不能太分散
theta = 5
for idx, I_ in enumerate(list(trans_dict_2.values())):
    m.addConstrs((gp.quicksum(y.get((i, j, k, t), 0)
                            for i in I_) <= theta
                for j in J
                for k in K
                for t in Y), "C5"+str(idx))

In [31]:
# 6. 单个地块 （含⼤棚 种植 的 ⾯积不宜太⼩，等等

delta = 1 / 3
m.addConstrs((x[i, j, k, t] >= a[i] * y[i, j, k, t] * delta
                for i, j, k, t in V), "C6")

m.addConstrs((x_s[I0, j, k, t] == l_s[I0, j, k, t] * x.sum(trans_dict_2[I0], j, k, t) * mu_data[I0, j, k, t] +(1 - l_s[I0, j, k, t]) * sale_data.get((I0, j, k, t), 1e10)
                for I0, j, k, t in x_s), "C71")
m.addConstrs(((l_s[I0, j, k, t] == 1) >> (x.sum(trans_dict_2[I0], j, k, t) * mu_data[I0, j, k, t] <=sale_data.get((I0, j, k, t), 1e10))
                for I0, j, k, t in x_s), "C72")

m.addConstrs(((l_s[I0, j, k, t] == 0) >> (x.sum(trans_dict_2[I0], j, k, t) * mu_data[I0, j, k, t] >=sale_data.get((I0, j, k, t), 1e10))
                for I0, j, k, t in x_s), "C73")


{('平旱地', 1, 1, 2024): <gurobi.GenConstr *Awaiting Model Update*>,
 ('平旱地', 1, 1, 2025): <gurobi.GenConstr *Awaiting Model Update*>,
 ('平旱地', 1, 1, 2026): <gurobi.GenConstr *Awaiting Model Update*>,
 ('平旱地', 1, 1, 2027): <gurobi.GenConstr *Awaiting Model Update*>,
 ('平旱地', 1, 1, 2028): <gurobi.GenConstr *Awaiting Model Update*>,
 ('平旱地', 1, 1, 2029): <gurobi.GenConstr *Awaiting Model Update*>,
 ('平旱地', 1, 1, 2030): <gurobi.GenConstr *Awaiting Model Update*>,
 ('平旱地', 2, 1, 2024): <gurobi.GenConstr *Awaiting Model Update*>,
 ('平旱地', 2, 1, 2025): <gurobi.GenConstr *Awaiting Model Update*>,
 ('平旱地', 2, 1, 2026): <gurobi.GenConstr *Awaiting Model Update*>,
 ('平旱地', 2, 1, 2027): <gurobi.GenConstr *Awaiting Model Update*>,
 ('平旱地', 2, 1, 2028): <gurobi.GenConstr *Awaiting Model Update*>,
 ('平旱地', 2, 1, 2029): <gurobi.GenConstr *Awaiting Model Update*>,
 ('平旱地', 2, 1, 2030): <gurobi.GenConstr *Awaiting Model Update*>,
 ('平旱地', 3, 1, 2024): <gurobi.GenConstr *Awaiting Model Update*>,
 ('平旱地', 3

In [32]:
# 4.3 ⽬标函数
obj = gp.LinExpr()
for t in Y:
    for I0, j, k in pl:
        if I0 == "平旱地":
            p_c = jia_data_l[I0, j, k, t] * 0.75 + 0.25 * jia_data_u[I0, j, k,t]
        if I0 == "梯⽥":
            p_c = jia_data_l[I0, j, k, t] * 0.75 + 0.25 * jia_data_u[I0, j, k,t]
        if I0 == "⼭坡地":
            p_c = jia_data_l[I0, j, k, t] * 0.75 + 0.25 * jia_data_u[I0, j, k,t]
        if I0 == "⽔浇地":
            p_c = jia_data_l[I0, j, k, t] * 0.75 + 0.25 * jia_data_u[I0, j, k,t]
        if I0 == "普通⼤棚":
            p_c = jia_data_l[I0, j, k, t] * 0.75 + 0.25 * jia_data_u[I0, j, k,t]
        if I0 == "智慧⼤棚":
            p_c = jia_data_l[I0, j, k, t] * 0.75 + 0.25 * jia_data_u[I0, j, k,t]
                
        c_c = ch_data[I0, j, k, t]
        current = p_c * x_s[I0, j, k, t] - c_c * x.sum(trans_dict_2[I0], j, k,t)
        obj += current

m.setObjective(obj, gp.GRB.MAXIMIZE)

In [33]:
 # 5. 模型求解
 m.write("model.lp")
 m.setParam(gp.GRB.Param.MIPGap, 0.01)
 m.setParam(gp.GRB.Param.TimeLimit, 120)
 m.optimize()

Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 120
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 23656 rows, 16618 columns and 56994 nonzeros
Model fingerprint: 0xf6d9ac20
Model has 875 quadratic constraints
Model has 1750 general constraints
Variable types: 8309 continuous, 8309 integer (8309 binary)
Coefficient statistics:
  Matrix range     [2e-01, 9e+01]
  QMatrix range    [1e+02, 2e+04]
  QLMatrix range   [1e+00, 1e+10]
  Objective range  [1e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-01, 9e+01]
  QRHS range       [8e+02, 1e+10]
  GenCon rhs range [8e+02, 1e+10]
  GenCon coe range [1e+02, 2e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 5351 rows and 3332 columns

In [37]:
# 6. 结果输出
xv = m.getAttr("X", x)
yv = m.getAttr("X", y)
# for k, v in xv.items():
# if v < .001 or k[3] > 2024:
# continue
# print(k, ":", v)

workbook = load_workbook(r"C:\Users\86181\Desktop\数模作业、笔记\农作物种植策略\results\result1_1.xlsx")
for t in Y:
    data_r = pd.read_excel(r"C:\Users\86181\Desktop\数模作业、笔记\农作物种植策略\results\result1_1.xlsx", sheet_name=str(t))
    # writer = xlsxwriter.Workbook('existing_file.xlsx', engine='openpyxl')
    worksheet_writer = workbook[str(t)]
    for index, row in data_r.iterrows():
        r_data = []
        for cindex, col_name in enumerate(data_r.columns[2:]):
            cc_name = col_name.strip()
            if (data_r["地块名"][index], J_d[cc_name], K[0] if index <= 53 else K[1], t) in xv:
                 vv = xv[data_r["地块名"][index], J_d[cc_name], K[0] if index <= 53 else K[1], t]
                 if vv > 0.001:
                     worksheet_writer.cell(index + 2, 3 + cindex, vv)

                     file_path =r"C:\Users\86181\Desktop\数模作业、笔记\农作物种植策略\results\result1_2.xlsx"
                     if os.path.exists(file_path):
                         os.remove(file_path)
                     workbook.save(file_path)

C:\Users\86181\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
